<a href="https://colab.research.google.com/github/EmoDetect/emodetect/blob/main/EmotionsKids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
### CNN models ###
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.utils import np_utils
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import models
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score

Parameters

In [ ]:
batch_size = 16
num_epochs = 50
input_shape = (48, 48, 1)
validation_split = .2
verbose = 1
num_classes = 7
base_path = 'models/'
shape_x = 48
shape_y = 48
image_size=(48,48)
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Data

In [ ]:
data = pd.read_csv('./data/fer2013/fer2013.csv')
data = data.head(500)
print(len(data))

In [ ]:
data['pixels']=data['pixels'].astype("string")
# print(data['pixels'])
pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []

# iterate through all pixels and create a matrix(face) of size 48 x 48
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.strip().split(' ',48*48)]
    if len(face) == 2304:
      face = np.asarray(face).reshape(width, height)
      face = cv2.resize(face.astype('uint8'),image_size)
      faces.append(face.astype('float32'))
# last = faces[-1]
# faces.append(last)
# del faces[-1]
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

faces /= 127.5
faces -= 1.

emotions = pd.get_dummies(data['emotion']).to_numpy()

In [ ]:
print(len(faces))

In [ ]:
datagen = ImageDataGenerator(
        zoom_range=0.2,          # randomly zoom into images
        rotation_range=10,       # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,   # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,    # randomly flip images
        vertical_flip=False)     # randomly flip images

In [ ]:
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.3,shuffle=True)
xval,xtest,yval,ytest=train_test_split(xtest,ytest,test_size=0.3,shuffle=True)

CNN

In [ ]:
def CNN():
    model = Sequential(name='CNN')
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    return model

In [ ]:
CNN=CNN()

Training

In [ ]:
early_stop = EarlyStopping('val_loss', patience=100)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=25, min_lr=0.00001,model='auto')
trained_models_path = base_path + 'CNN'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [ ]:
CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
CNN_history =CNN.fit(datagen.flow(xtrain, ytrain, batch_size),
          steps_per_epoch=len(xtrain) / batch_size, 
          epochs=num_epochs, 
          verbose=1, 
          callbacks=callbacks,
          validation_data=(xval,yval))

In [ ]:
fig,axes=plt.subplots(2,2,figsize=(20, 20))
for (m,history), ax in zip({'CNN':CNN_history}.items(),axes):
    # Loss Curves
    
    ax[0].plot(history.history['loss'],'r',linewidth=2.0)
    ax[0].plot(history.history['val_loss'],'b',linewidth=2.0)
    ax[0].legend(['Training loss', 'Validation Loss'],fontsize=18)
    ax[0].set_xlabel('Epochs ',fontsize=16)
    ax[0].set_ylabel('Loss',fontsize=16)
    ax[0].set_title('Loss Curves '+m,fontsize=16)
 
    # Accuracy Curves
    ax[1].plot(history.history['accuracy'],'r',linewidth=2.0)
    ax[1].plot(history.history['val_accuracy'],'b',linewidth=2.0)
    ax[1].legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    ax[1].set_xlabel('Epochs ',fontsize=16)
    ax[1].set_ylabel('Accuracy',fontsize=16)
    ax[1].set_title('Accuracy Curves '+m,fontsize=16)

In [ ]:
ypred=CNN.predict(xtest)
ypred_=np.argmax(ypred, axis=1)
ytest_=np.argmax(ytest, axis=1)
print(classification_report(ytest_, ypred_,digits=3))

In [ ]:
# import itertools
# from sklearn.metrics import confusion_matrix
# from matplotlib.pyplot import figure


# fig = figure(figsize=(10, 10))

# ypred=CNN.predict(xtest[:1])
# rounded_predections=np.argmax(ypred, axis=1)
# print(rounded_predections)
# rounded_labels=np.argmax(ytest, axis=1)
# print(rounded_labels)

# cm = confusion_matrix(rounded_labels, rounded_predections)
# title='Confusion matrix '+CNN.name
    

# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title(title)
# plt.colorbar()
# tick_marks = np.arange(len(labels))
# plt.xticks(tick_marks, labels, rotation=45)
# plt.yticks(tick_marks, labels)
# fmt = 'd'
# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], fmt),
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")
    
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.tight_layout()
# plt.show()

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, request
from flask_restful import Resource, Api

app = Flask(__name__)

@app.route('/emotion')
def get_emotion():
    return {'hello': 'world'}


@app.route('/postImage', methods=['POST'])
def post_image():
    if request.method == 'POST':
       return convert_image(request.get_json())

def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.get('/shutdown')
def shutdown():
    shutdown_server()
    return 'Server shutting down...'


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
def convert_image(encoded_image):
    from PIL import Image
    import base64
    import io
    import numpy as np
    import torch

    base64_decoded = base64.b64decode(encoded_image)

    image = Image.open(io.BytesIO(base64_decoded))
    pixels_image = image.getdata()
    pixels_image = list(pixels_image)

    pixels_list = []

    for tuple in pixels_image:
        gray = tuple[0] * 0.299 + tuple[1] * 0.587 + tuple[2] * 0.114
        pixels_list.append(gray)

    print(len(pixels_list))

    width, height = 1024, 768
    faces = []

    face = pixels_list

    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'),image_size)
    faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)

    import itertools
    from sklearn.metrics import confusion_matrix
    from matplotlib.pyplot import figure


    fig = figure(figsize=(10, 10))

    ypred=CNN.predict(faces)
    rounded_predections=np.argmax(ypred, axis=1)
    return rounded_predections[0]

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
from matplotlib.pyplot import figure


fig = figure(figsize=(10, 10))

ypred=CNN.predict(faces)
rounded_predections=np.argmax(ypred, axis=1)
print(rounded_predections)
# rounded_labels=np.argmax(ytest, axis=1)
# print(rounded_labels)